In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv(r"C:\Users\SIDDHARTH\Downloads\spotify_millsongdata.csv")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [3]:
df.shape

(57650, 4)

In [4]:
df = df.sample(20000).drop('link',axis='columns')
df.head()

,artist,song,text
23911,America,Stereo,Stereo \r\nWe hear both sides \r\nWe sympath...
18881,Stevie Wonder,Another Star,La la la la la la lala la \r\nLa la la la la ...
44747,Nat King Cole,For All We Know,For all we know \r\nWe may never meet again ...
47837,Pearl Jam,Nothing As It Seems,Don't feel like home \r\nEase a little out \...
23698,Allman Brothers Band,I'm Gonna Move To The Outskirts Of Town,Well I'm gonna move baby \r\nWell on the outs...


In [5]:
df['text'].iloc[0]

"Stereo  \r\nWe hear both sides  \r\nWe sympathize  \r\nWe live our lives in stereo  \r\nThe left and the right  \r\nThe dark and the light  \r\nWe wrestle with the balance  \r\nWe change our tone  \r\nWe leave our phone  \r\nAnd tape record our absence  \r\nIn lovin' memory  \r\n  \r\nStereo  \r\nWe're livin' it in stereo  \r\nWe fix it so our love is high fidelity  \r\nMix it so we never lose the melody  \r\nWe try to equalize our lives in stereo  \r\n  \r\nSo on we go  \r\nFrom side to side  \r\nAs we divide  \r\nA single life in stereo  \r\nThe far and the wide  \r\nWe override  \r\nThe feedback from the others  \r\nThe unkind phrase  \r\nWe lock in phase  \r\nWe're only really listening to the stereo  \r\n  \r\nStereo  \r\nWe're livin' it in stereo  \r\nWe tune it till we have a perfect parody  \r\nCommune with such a fine-cut, crystal clarity  \r\nIt seems to symbolize  \r\nOur lives in stereo  \r\n  \r\nOo (la, la, la, la, la, la), yeah, yeah, yeah  \r\nOo (la, la, la, la, la, l

In [6]:
df.isna().sum()

artist    0
song      0
text      0
dtype: int64

In [7]:
df.duplicated().sum()

0

In [8]:
df.head()

,artist,song,text
23911,America,Stereo,Stereo \r\nWe hear both sides \r\nWe sympath...
18881,Stevie Wonder,Another Star,La la la la la la lala la \r\nLa la la la la ...
44747,Nat King Cole,For All We Know,For all we know \r\nWe may never meet again ...
47837,Pearl Jam,Nothing As It Seems,Don't feel like home \r\nEase a little out \...
23698,Allman Brothers Band,I'm Gonna Move To The Outskirts Of Town,Well I'm gonna move baby \r\nWell on the outs...


In [9]:
def preprocess_text(text):
    # Convert all words to lowercase
    text = text.lower()
    # Remove punctuation marks and extra whitespaces
    text = re.sub(r'[^\w\s]', '', text)
    # Replace multiple whitespaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Strip leading and trailing whitespaces
    text = text.strip()
    return text

In [10]:
df['text'] = df['text'].apply(preprocess_text)

In [11]:
df['text'].iloc[0]

'stereo we hear both sides we sympathize we live our lives in stereo the left and the right the dark and the light we wrestle with the balance we change our tone we leave our phone and tape record our absence in lovin memory stereo were livin it in stereo we fix it so our love is high fidelity mix it so we never lose the melody we try to equalize our lives in stereo so on we go from side to side as we divide a single life in stereo the far and the wide we override the feedback from the others the unkind phrase we lock in phase were only really listening to the stereo stereo were livin it in stereo we tune it till we have a perfect parody commune with such a finecut crystal clarity it seems to symbolize our lives in stereo oo la la la la la la yeah yeah yeah oo la la la la la la yeah yeah yeah oo la la la laa stereo stereo we fix it so our love is high fidelity mix it so we never lose the melody we try to equalize our lives in stereo stereo stereo we fix it so our love is high fidelity 

In [13]:
ps = PorterStemmer()

In [14]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [15]:
df.shape

(20000, 3)

In [16]:
df['text'] = df['text'].apply(stem)

In [17]:
df['text'].iloc[0]

'stereo we hear both side we sympath we live our live in stereo the left and the right the dark and the light we wrestl with the balanc we chang our tone we leav our phone and tape record our absenc in lovin memori stereo were livin it in stereo we fix it so our love is high fidel mix it so we never lose the melodi we tri to equal our live in stereo so on we go from side to side as we divid a singl life in stereo the far and the wide we overrid the feedback from the other the unkind phrase we lock in phase were onli realli listen to the stereo stereo were livin it in stereo we tune it till we have a perfect parodi commun with such a finecut crystal clariti it seem to symbol our live in stereo oo la la la la la la yeah yeah yeah oo la la la la la la yeah yeah yeah oo la la la laa stereo stereo we fix it so our love is high fidel mix it so we never lose the melodi we tri to equal our live in stereo stereo stereo we fix it so our love is high fidel mix it so we never lose the melodi we tr

In [18]:
ps.stem('something')

'someth'

In [19]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')

In [20]:
vectors = tfidvector.fit_transform(df['text'])

In [21]:
vectors.shape

(20000, 40455)

In [22]:
vectors[0]

<1x40455 sparse matrix of type '<class 'numpy.float64'>'
	with 56 stored elements in Compressed Sparse Row format>

In [23]:
print(tfidvector.get_feature_names_out())

['00' '000' '007' ... 'zyada' 'zydeco' 'zydin']


In [24]:
similarity = cosine_similarity(vectors)

In [25]:
similarity[0]

array([1.        , 0.45858595, 0.00460524, ..., 0.01462682, 0.01505505,
       0.01953462])

In [26]:
similarity.shape

(20000, 20000)

In [27]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:6]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [28]:
# recommendation("Down On The River")

In [29]:
pickle.dump(df.to_dict(),open('songs.pkl','wb'))

In [30]:
pickle.dump(similarity,open('similarity.pkl','wb'))